In [1]:
from pathlib import Path
import pandas as pd
import os 
import json

In [2]:
DRIVE_PATH = Path("/mnt/h/")
KODF_PATH = DRIVE_PATH / "kodf_release"
DFDC_PATH = DRIVE_PATH / "dfdc_extracted"

# Pre-Proc KODF

In [3]:
# Uncomment for processing 
from deepfake_detection.utils.defaults import RANDOM_STATE

from sklearn.utils import shuffle

def get_kodf_train_test_people(unique_persons: list[str]) -> tuple[list[str], list[str]]:
    shuffled_people = shuffle(unique_persons, random_state=RANDOM_STATE)
    print(shuffled_people)
    
    return None, None
kodf_synthesized_metadata = pd.read_csv(KODF_PATH / "synthesized_video_metadata.csv")
kodf_synthesized_videos_to_keep = kodf_synthesized_metadata[~kodf_synthesized_metadata.model.isin(["fo", "audio-driven"])]
kodf_synthesized_videos_to_keep['video_path'] = KODF_PATH.relative_to(DRIVE_PATH) / "synthesized_videos" / kodf_synthesized_videos_to_keep['model'] / kodf_synthesized_videos_to_keep['date_time'].str.replace("-", "") / kodf_synthesized_videos_to_keep['target_person_id'] / kodf_synthesized_videos_to_keep['video']
kodf_synthesized_videos_to_keep['video_path'] = kodf_synthesized_videos_to_keep['video_path'].apply(lambda x: str(x))
kodf_synthesized_videos_to_keep['label'] = "FAKE"
kodf_synthesized_videos_to_keep.drop(["sex", "date_time", "model", "target_person_id", "target_video"], axis=1, inplace=True)
kodf_synthesized_videos_to_keep.rename(columns={"source_person_id": "person_id"}, inplace=True)
kodf_synthesized_videos_to_keep.set_index("video", inplace=True)

kodf_original_metadata = pd.read_csv(KODF_PATH / "original_video_metadata.csv")
kodf_original_metadata['video_path'] = KODF_PATH.relative_to(DRIVE_PATH) / "original_videos" / kodf_original_metadata["person_id"] / kodf_original_metadata["video"]
kodf_original_metadata['video_path'] = kodf_original_metadata['video_path'].apply(lambda x: str(x))
kodf_original_metadata['label'] = "REAL"
kodf_original_metadata.drop(["sex", "date_time", "studio"], axis=1, inplace=True)
kodf_original_metadata.set_index("video", inplace=True)

kodf_combined = pd.concat([kodf_synthesized_videos_to_keep, kodf_original_metadata])

train_people, test_people = get_kodf_train_test_people(kodf_combined['person_id'].unique())

# bad_paths = []

# for path in kodf_combined['video_path']:
#     if not os.path.exists(DRIVE_PATH / path):
#         print(f"Bad path: {path}")
#         bad_paths.append(path)
    
# kodf_combined = kodf_combined[~kodf_combined['video_path'].isin(bad_paths)]
# kodf_dict = kodf_combined.to_dict(orient="index")

# with open(KODF_PATH / "master_metadata.json", 'w') as f:
#     json.dump(kodf_dict, f)

ModuleNotFoundError: No module named 'deepfake_detection'

In [ ]:
# Uncomment for processing 

parts = [i for i in range(50)]
master_metadata = {}
for part in parts:
    print(f"Processing Part: {part}")
    folder_name = f"dfdc_train_part_{str(part).zfill(2)}"
    inner_folder_name = f"dfdc_train_part_{part}"
    current_part_path = DFDC_PATH / folder_name / inner_folder_name
    metadata_path = current_part_path / "metadata.json" 
    part_meta = {}
    with open(metadata_path) as f:
        meta_data = json.load(f)
    for video, data in meta_data.items():
        video_path = current_part_path / video
        if not os.path.exists(video_path) == True:
            print("CURRENT VIDEO PATH", video_path, video, data)
            continue
        del data['split']
        if 'original' in data.keys():
            del data['original']

        data['video_path'] = str(video_path.relative_to(DRIVE_PATH))
        part_meta[video] = data
    master_metadata.update(part_meta)

with open(DFDC_PATH / "master_metadata.json", 'w') as f:
    json.dump(master_metadata, f)

In [ ]:
kodf_df = pd.read_json(KODF_PATH / "master_metadata.json", orient="index")
dfdc_df = pd.read_json(DFDC_PATH / "master_metadata.json", orient="index")
kodf_df

In [ ]:
final_dataset = pd.concat([kodf_df, dfdc_df])
final_dataset.reset_index(inplace=True)
final_dataset.columns = ["video", "video_path", "label"]
final_dataset.to_csv("metadata.csv", index=False)